<a href="https://colab.research.google.com/github/MLcmore2023/MLcmore2023/blob/main/day2_pm_afternoon/decision-tree-demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Intro to Pandas syntax

In [511]:
import pandas as pd
dataset = pd.DataFrame({'Name': ['Alice', 'Bob', 'Charlie', 'David', 'Eva'],
        'Age': [25, 30, 22, 28, 35],
        'City': ['New York', 'Los Angeles', 'Chicago', 'Houston', 'Miami']})
display(dataset)

,Name,Age,City
0,Alice,25,New York
1,Bob,30,Los Angeles
2,Charlie,22,Chicago
3,David,28,Houston
4,Eva,35,Miami


`iloc` lets us select specific row or rows from the dataset

In [512]:
# get row 2
row_2 = dataset.iloc[2]
print(row_2)

Name    Charlie
Age          22
City    Chicago
Name: 2, dtype: object


In [513]:
# get row 1, 3, and 4
row_1_3_4 = dataset.iloc[ [1,3,4] ]
print(row_1_3_4)

    Name  Age         City
1    Bob   30  Los Angeles
3  David   28      Houston
4    Eva   35        Miami


To get a column, we use the column name

In [514]:
column_2 = dataset["Age"]
print(column_2)

0    25
1    30
2    22
3    28
4    35
Name: Age, dtype: int64


we can perform element-wise comparisons on pandas column like this:

In [515]:
older_than_25 = dataset["Age"] > 25
print(older_than_25)

0    False
1     True
2    False
3     True
4     True
Name: Age, dtype: bool


In [516]:
# we can use the column of True/False to easily filter a table to get only people older than 25
filtered_dataset = dataset[older_than_25]
display(filtered_dataset )

,Name,Age,City
1,Bob,30,Los Angeles
3,David,28,Houston
4,Eva,35,Miami


#### Numpy syntax quick recap


In [517]:
import numpy as np

In [518]:
number_array = np.array( [1, 1, 0, 1, 0])
print( np.sum(number_array) )

# in numpy, we can add up True/False. True's are 1, False's are 0
boolean_array = np.array( [True, True, False, True, False])
print( np.sum(boolean_array) )

3
3


we can perform element-wise comparisons on numpy array like this:

In [519]:
array1 = np.array([1,2,3,4,5,6,7,8,9,10])

indices_of_elements_larger_than_3 = ( array1>3 )
print(indices_of_elements_larger_than_3 )

[False False False  True  True  True  True  True  True  True]


In [520]:
indices_of_even_elments = ( array1 % 2 == 0 )
print(indices_of_even_elments  )

[False  True False  True False  True False  True False  True]


adding `~` in front means NOT, which flips the True and False

In [521]:
print(~indices_of_even_elments  )

[ True False  True False  True False  True False  True False]


We can also use the `argwhere` method to find all indices which some condition is true

In [522]:
indices_of_elements_larger_than_3 = np.argwhere( array1 > 3).flatten()
print(indices_of_elements_larger_than_3)

[3 4 5 6 7 8 9]


this allows us to quickly count how many items mets certain condition

In [523]:
count_of_elements_larger_than_3 = sum(indices_of_elements_larger_than_3)
print(count_of_elements_larger_than_3)

42


In [524]:
count_of_even_elments = sum(indices_of_even_elments)
print(count_of_even_elments)

5


We can also lookup a list of indices in an array

In [525]:
array2 = np.array(['a','b','c','d','e','f','g'])
indices = np.array([0,1,5,6])

print(array2[indices]) # the 0th, 1th, 5th, and 6th item

['a' 'b' 'f' 'g']


# Decision Tree
Decision tree is a supervised machine learning algorithm used for classification and regression. A binary tree is constructed, where each internal node represents a feature, and each leaf node represents a prediction or outcome. The tree segment the predictor space (see image below). The tree is built by recursively splitting the dataset based on the best attribute that maximizes information gain or minimizes impurity. When a prediction is needed, the input record traverses the tree from the root to a leaf, following the path determined by the attribute values, and the corresponding prediction is made based on the outcome associated with that leaf node.

The classical decision tree algorithms have been around for decades, and they are the basis of more powerful techniques such as random forest.


<img src="https://miro.medium.com/v2/resize:fit:640/format:webp/1*AqgTZMxAavMAXnDiJtZbBQ.png">

<img src="https://miro.medium.com/v2/resize:fit:640/format:webp/1*FrktyoWBlqyh8-J0DtBexA.png">


### Import libraries

In [526]:
import numpy as np
import pandas as pd

### Read data from CSV
We will use the Height-Weight dataset in this tutorial. Gender, height, and weight will be used to predict body mass `Index` or whether a person is `medically_obese`.  

In [527]:
data = pd.read_csv("https://raw.githubusercontent.com/chriswmann/datasets/master/500_Person_Gender_Height_Weight_Index.csv")
data['medically_obese'] = (data.Index >= 4).astype('int') # anyone with index >= 4 will be considered as medically_obese
display(data)

,Gender,Height,Weight,Index,medically_obese
0,Male,174,96,4,1
1,Male,189,87,2,0
2,Female,185,110,4,1
3,Female,195,104,3,0
4,Male,149,61,3,0
...,...,...,...,...,...
495,Female,150,153,5,1
496,Female,184,121,4,1
497,Female,141,136,5,1
498,Male,150,95,5,1


here we split the data into input variable (y) and output variables (X).



In [528]:
#Split the dataset into X (input features) and y (output labels)
feature_list = ['Gender', 'Height', 'Weight']
X = data[ ['Gender', 'Height', 'Weight'] ]
y = data['medically_obese']

In [529]:
X

,Gender,Height,Weight
0,Male,174,96
1,Male,189,87
2,Female,185,110
3,Female,195,104
4,Male,149,61
...,...,...,...
495,Female,150,153
496,Female,184,121
497,Female,141,136
498,Male,150,95


In [530]:
y

0      1
1      0
2      1
3      0
4      0
      ..
495    1
496    1
497    1
498    1
499    1
Name: medically_obese, Length: 500, dtype: int64

we now split the dataset into training and testing sets, using the sklearn library's `train_test_split` function

In [531]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# convert y_train and y_test to numpy arrays, since they are just list of numbers
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

## Decision tree concept

The process of building a decision tree mainly involves two steps:
1. Dividing the predictor space into several distinct, non-overlapping regions
2. Using the most-common class label for the region of new observation as the prediction of the new observation

<img src="https://miro.medium.com/v2/resize:fit:640/format:webp/1*FrktyoWBlqyh8-J0DtBexA.png">

In order to split the predictor space into distinct regions, we use binary recursive splitting, which grows our decision tree until we reach a stopping criterion.

<img src="https://miro.medium.com/v2/resize:fit:640/format:webp/1*AqgTZMxAavMAXnDiJtZbBQ.png">

In a decision tree, every node will have two children. The children can be leaf nodes or sub-trees, which enhances the prediction of the previous node. This goes until we get to a node that does not split. This last node is known as a leaf node.

For example:

<img src="https://raw.githubusercontent.com/MLcmore2023/MLcmore2023/main/.images/decisiontree1.png">

Our algorithm's goal is to find the best variables (features) and cutoff values for each decision node

### Cost function
"Impurty" means the likelyhood of an incorrect classification when a division is performed. The purpose of cost functions in this algorithm is to evaluate impurity.

For example, lets make a cut at weight=100kg. This classifies anyone above 100kg as medically obese. If we do so, there will be 82 miss classifications

In [532]:
# prediction by using 100kg as cutting point
prediction_using_100kg = data['Weight'] >= 100
print(prediction_using_100kg)

0      False
1      False
2       True
3       True
4      False
       ...  
495     True
496     True
497     True
498    False
499     True
Name: Weight, Length: 500, dtype: bool


In [533]:
true_label = (data['medically_obese']==1)

In [534]:
#Count of misclassifications when cutting at 100kg:

# counts of entries where our prediction gives True, but the true label is False
false_positives = sum(  prediction_using_100kg & ~true_label  )

# counts of entries where our prediction gives False, but the true label is True
false_negatives = sum(  ~prediction_using_100kg & true_label )

print(false_positives, false_negatives)
print("Count of misclassifications:",false_positives + false_negatives)

18 64
Count of misclassifications: 82


If we cut at weight=80kg, there will be only 78 misclassifications. Therefore, this cut is better, albeit still a really bad prediction.

In [535]:
#Count of misclassifications when cutting at 80kg:
prediction_using_80kg = data['Weight'] >= 80

false_positives = sum(  prediction_using_80kg & ~true_label  )
false_negatives = sum(  ~prediction_using_80kg & true_label )
print(false_positives, false_negatives)
print("Count of misclassifications:",false_positives + false_negatives)

63 15
Count of misclassifications: 78


### Gini index
The Gini index (also known as Gini impurity) is a popular cost function in decision trees. It is a measure that quantifies the impurity or disorder of a dataset with respect to the class labels.
The Gini index ranges from 0 to 0.5, where 0 indicates perfect purity (all data points belong to a single class) and 0.5 indicates maximum impurity (data points are evenly distributed across all classes). The Gini index is calculated as follows:

$$
Gini = 1 - \sum_{i=1}^{n} (P_i)^2
 $$

Where P<sub>i</sub> is the probability of having that class or value.

In [536]:
def gini_index(y_list, print_demo=False):
  # find the list of categories in the list (which is the list of unique elements in the list)
  category_list = np.unique(y_list)

  # number of total elements in the list
  N = len(y_list)

  gini = 1
  for category in category_list: # go through all categories
    count = np.sum(y_list==category)
    P_i = count / N
    if print_demo:
      print(f"Category: {category}, Count: {count}, P_i: {P_i}")

    gini = gini - P_i**2
  return gini

# we can see that the result is very close to 0.5 because the list is very impure
print( gini_index(data.Gender.values,   print_demo=True) )

Category: Female, Count: 255, P_i: 0.51
Category: Male, Count: 245, P_i: 0.49
0.4998


### Entropy
Another way to measure impurity is entropy. In information theory, entropy describes the average level of information or uncertainty and can be defined as the following.
$$ S = -\sum P_i \log_2 P_i$$
Unlike the Gini index, whose range goes from 0 to 0.5, the entropy range is different, since it goes from 0 to 1. Values close to zero are less impure than those that approach 1.

Later, when making a decision tree split, entropy is used to evaluate the potential splits based on different features. The split that results in the lowest entropy (i.e., the greatest reduction in impurity) is chosen.

In [537]:
def entropy(y_list, print_demo=False):
    # find the list of categories in the list (which is the list of unique elements in the list)
    category_list = np.unique(y_list)

    # number of total elements in the list
    N = len(y_list)

    entropy_val = 0
    for category in category_list: # go through all categories
        count = np.sum(y_list == category)
        P_i = count / N

        if print_demo:
            print(f"Category: {category}, Count: {count}, P_i: {P_i}")

        entropy_val = entropy_val - P_i * np.log2(P_i)

    return entropy_val

# we can see that the result is very close to 1 because the list is very impure
print( entropy(data.Gender.values)  )

0.9997114417528099


In [538]:
def create_split(array, threshold):

    # Find all indices of elements in array that are below the threshold
    left_indices = np.argwhere(array <= threshold).flatten()

    # Find all indices of elements in array that are above the threshold
    right_indices = np.argwhere(array > threshold).flatten()

    return left_indices, right_indices

example_list = np.array([14, 58, 84, 23, 99, 73])
create_split(example_list, threshold=50)

(array([0, 3]), array([1, 2, 4, 5]))

### Information Gain

As we have seen, cuts are compared by impurity. Therefore, we are interested in comparing those cuts that generate less impurity. We use information gain for this. It indicates the improvement when making different partitions and is usually used with entropy (it could also be used with the Gini index, although in that case the name would not be called information gain).

Information gain is calculated by measuring the reduction in entropy (or increase in purity) achieved after splitting the data based on a specific feature. It is calculated as the difference between the entropy of the parent node before the split and the weighted average of the entropies of the child nodes after the split.

$$ \text{Information Gain}(D, A) = \text{Entropy}(D) - \sum_{v \in \text{Values}(A)} \frac{|D_v|}{|D|} \times \text{Entropy}(D_v)
$$

$
\begin{align*}
D &: \text{dataset at the current node.} \\
A &: \text{feature being considered for splitting.} \\
\text{Values}(A) &: \text{possible values that feature } A \text{ can take.} \\
D_v &: \text{subset of data where feature } A \text{ takes the value } v.
\end{align*}
$

The higher the Information Gain for a feature, the more it helps in reducing uncertainty in the target variable. Later, our algorithm will select the feature that offers the highest Information Gain as the splitting criterion at each node.


In [539]:
def information_gain(X_one_feature_only, y, threshold):
    # calculate the entropy before the split
    parent_entropy = entropy(y)

    # find the indices of elements above and below the threshold so we can make the split
    left_indices, right_indices = create_split(X_one_feature_only, threshold)

    left_elements = y[left_indices]
    right_elements = y[right_indices]

    # find how many elements are there before and after the split
    n = len(y)
    n_left = len(left_indices)
    n_right = len(right_indices)

    # if the split resulted in 0 item in either side (meaning all elements went into one side and the other side is empty)
    # then the information gain is zero because the split is useless (not a split at all)
    if n_left == 0 or n_right == 0:
        return 0

    # calculate the entropy of two sides
    left_child_entropy = entropy(left_elements)
    right_child_entropy = entropy(right_elements)

    # take the weighted average
    child_entropy = (n_left/n)*left_child_entropy + (n_right/n)*right_child_entropy

    return parent_entropy - child_entropy

# example of a perfect cut:
X_example = np.array([1,2,3,4,5,  6,7,8,9,10]) # example feature
y_example = np.array([0,0,0,0,0,  1,1,1,1,1]) # two classes
information_gain(X_example, y_example, 5)

1.0

An information gain of 1 would be the best possible result, meaning there is a perfect separation of the target variable into its classes. This means that after the split, each subset contains only instances of a single class, completely reducing the uncertainty. In practical scenarios, achieving an information gain of 1 is uncommon, because real-world data that have noise, overlapping classes, or other complexities.

Here we calculate the information gain of splitting the dataset by threshold of `weight=100kg`

In [540]:
X_weight = data['Weight'].to_numpy()
y = data['medically_obese'].to_numpy()
information_gain(X_weight,y , 100)

0.3621623817673466

### Finding the best variable and threshold for making splits

To make a split, we have 2 steps:
1. Calculate the information gain for possible threshold values of all features (variables).
2. Choose the split that generates the highest information gain as a split.


In [541]:
def max_information_gain_split(X, y, feature_list):
    # initialize the max info gain, feature, and threshold as void values
    best_info_gain = -1
    best_feature =  None
    best_threshold = None

    # go over all features
    for feature in feature_list:
        # X_of_feature will be an array of all values of the selected feature
        X_of_feature = X[feature]
        # convert the pandas dataset column to numpy array since the information_gain function takes in arrays
        X_of_feature = X_of_feature.to_numpy()

        # we use `np.unique` to find all possible values which we can use as the threshold, and go over each one
        possible_threshold_options = np.unique(X_of_feature)
        for threshold in possible_threshold_options:
            # calculate the information gain of this specific threshold for this specific feature
            info_gain = information_gain(X_of_feature, y, threshold)

            # keep this as the `best` one if it have info_gain higher than the existing best_info_gain
            if  info_gain > best_info_gain:
                best_info_gain = info_gain
                best_feature =  feature
                best_threshold = threshold

    return best_feature, best_threshold, best_info_gain

best_feature, best_threshold, best_info_gain = max_information_gain_split(X,y,feature_list)

print(f"The best split for {best_feature} = {best_threshold}. The info gain is {best_info_gain}")

The best split for Weight = 102. The info gain is 0.3824541370911896


As we can see, the variable with the highest Information Gain is Weight. Therefore, it will be the variable that we first use to do the split. In addition, we also have the value on which the split must be performed: 102.

### Decision tree algorithm
The main algorithm can be divided into three steps:
1. Initialization of parameters (e.g. maximum depth, minimum samples per split)
2. Building the decision tree, involving binary recursive splitting, evaluating each possible split at the current stage, and continuing to grow the tree until a stopping criterion is satisfied
3. Making a prediction, which can be described as traversing the tree recursively and returning the most-common class label as a response value.

In our tree, there are two kinds of nodes: leaf nodes and subtrees. We will store them as python dictionaries.

In [542]:
example_leaf_node = {
    "is_leaf":True,
    "value": 0 # the most_common_Label here
}

example_subtree = {
    "is_leaf":False,
    "feature": 'Weight',
    "threshold": 103,
    "left": example_leaf_node, # can be any leaf node or subtree
    "right": {} # can be any leaf node or subtree
}


### Building a tree and determining the depth of the tree
We use binary recursive splitting to build a tree. We first evaluate every possible split for every feature to select the best place to split, and make the split. Then, we use recursion to repeat, continuing to grow the tree until we meet a stopping criterion.

Without a stopping mechanism in place, we would have created an endless loop which the program keeps on splitting forever. We will use three ways to limit the tree depth:

- `max_depth`: maximum depth of the tree.
- `min_samples_split`: indicates the minimum number of observations a sheet must have to continue creating new nodes.
- `min_information_gain`: the minimum amount the information gain must increase for the tree to continue growing.

Once we satisfy the stopping criteria the method will recursively return all nodes, allowing us to build a full-grown decision tree.


In [543]:
def build_tree(X, y, max_depth=100, min_samples_split=2, min_info_gain=0.00001,recursion_depth_counter=0):
    n_samples, n_features = X.shape
    n_class_labels = len(np.unique(y))

    # stopping criteria (max depth and min sample split conditions, or if all elements in dataset belongs to the same class)
    if ( (recursion_depth_counter > max_depth)
        or (X.shape[0] < min_samples_split)
        or (n_class_labels<=1) ):

        most_common_Label = np.argmax(np.bincount(y))
        return {"is_leaf":True, "value":most_common_Label}

    # get best split
    best_feat, best_thresh, best_info_gain = max_information_gain_split(X, y, feature_list)

    # stopping criteria (the best info gain is too small)
    """ NOT IMPLEMENTED. Left as an exercise for student """

    # make split
    left_idx, right_idx = create_split(X[best_feat].to_numpy(), best_thresh)

    # grow children recursively, by calling the build_tree function again
    left_child = build_tree(X.iloc[left_idx], y[left_idx], max_depth, min_samples_split, min_info_gain,recursion_depth_counter + 1)
    right_child = build_tree(X.iloc[right_idx], y[right_idx], max_depth, min_samples_split, min_info_gain, recursion_depth_counter + 1)
    return {"is_leaf":False, "feature": best_feat, "threshold":best_thresh, "left":left_child, "right":right_child }



max_depth = 5
min_samples_split = 20
min_information_gain = 0.0001

trained_tree = build_tree(X_train,y_train, max_depth, min_samples_split,min_information_gain)
display(trained_tree)

{'is_leaf': False,
 'feature': 'Weight',
 'threshold': 101,
 'left': {'is_leaf': False,
  'feature': 'Height',
  'threshold': 177,
  'left': {'is_leaf': False,
   'feature': 'Weight',
   'threshold': 72,
   'left': {'is_leaf': False,
    'feature': 'Height',
    'threshold': 150,
    'left': {'is_leaf': True, 'value': 0},
    'right': {'is_leaf': True, 'value': 0}},
   'right': {'is_leaf': False,
    'feature': 'Height',
    'threshold': 160,
    'left': {'is_leaf': False,
     'feature': 'Weight',
     'threshold': 78,
     'left': {'is_leaf': True, 'value': 1},
     'right': {'is_leaf': True, 'value': 1}},
    'right': {'is_leaf': False,
     'feature': 'Weight',
     'threshold': 81,
     'left': {'is_leaf': True, 'value': 0},
     'right': {'is_leaf': False,
      'feature': 'Weight',
      'threshold': 90,
      'left': {'is_leaf': True, 'value': 1},
      'right': {'is_leaf': True, 'value': 1}}}}},
  'right': {'is_leaf': True, 'value': 0}},
 'right': {'is_leaf': False,
  'feature

### Making Prediction using the tree
Making a prediction can be implemented by recursively traversing the tree. For every sample in our dataset, we compare the node feature and threshold values to the current sample's values and decide if we have to take a left or a right turn.

Once we reach a leaf node we simply return the most common class label as our prediction.

In [544]:
def traverse_tree(x, node):
    if node["is_leaf"]:
        return node["value"]

    feature = node["feature"]
    threshold =  node["threshold"]
    if x[feature] <= threshold:
        return traverse_tree(x, node["left"])
    else:
        return traverse_tree(x, node["right"])

def predict(x, decision_tree):
    return traverse_tree(x, decision_tree)

# predict the class for person #20
print(predict(X_test.iloc[20], trained_tree))

# the actual class for person #20
print(y_test[20])

1
1


### Evaluate accuracy

Finally, we run the `predict` function on all of the samples in the test dataset, and compare it againsst the actual class label.

In [545]:
correct_predictions = 0
total_predictions = len(y_test)

for i in range(len(y_test)):
    predicted_class = predict(X_test.iloc[i], trained_tree)
    actual_class = y_test[i]

    if predicted_class == actual_class:
        correct_predictions += 1

accuracy = correct_predictions / total_predictions
print(f"Accuracy: {accuracy}")

Accuracy: 0.94


### Exercise:
0. Re-run the code with `max_depth` set to 2. Then, re-run the code with `max_depth` set to 20. Write down how it affected the accuracy and the tree.
1. Modify code so it uses the Gini index in cost function instead of the entropy function. Write down how it affected the accuracy.
2. Is it possible for information gain of a split to be 0? if so, what does it mean and how should our program handle this case?
3. In our code, the `min_information_gain` stopping criterion is not implemented. Implement this by replacing `""" NOT IMPLEMENTED. Left as an exercise for student """` with code. When the minimum amount the information gain is too small, the tree should stop growing. (Hint: you may find the code for other stopping criterions useful)

## References
- https://towardsdatascience.com/implementing-a-decision-tree-from-scratch-f5358ff9c4bb
- https://anderfernandez.com/en/blog/code-decision-tree-python-from-scratch/
- https://towardsai.net/p/programming/decision-trees-explained-with-a-practical-example-fe47872d3b53
- https://www.datacamp.com/tutorial/decision-tree-classification-python
- https://towardsdatascience.com/decision-tree-algorithm-in-python-from-scratch-8c43f0e40173